In [6]:
import pandas as pd
import numpy as np
%matplotlib inline

In [ ]:
DATA_PATH = "../data/HSK Official With Definitions 2012 L1 freqorder.txt"

df = pd.read_csv(DATA_PATH, sep='\t', header=0, names=['simplificado', 'tradicional', 'pinyin_num', 'pinyin_carac', 'significado'])
df

In [19]:
DATA_PATH = "../data/"
NAME = "HSK Official With Definitions 2012 L{hsk_level:0d} freqorder.txt"
DATA_DICT = {i : NAME.format(hsk_level=i) for i in range (1, 7)} 
COLUNAS = ['simplificado', 'tradicional', 'pinyin_num', 'pinyin_carac', 'significado']
#df = pd.read_csv(DATA_PATH + DATA_LIST[0], sep='\t', header=0, names=['simplificado', 'tradicional', 'pinyin_num', 'pinyin_carac', 'significado'])
df_list = []
for hsk in DATA_DICT:
    aux = pd.read_csv(DATA_PATH + DATA_DICT[hsk], sep='\t', header=0, names=COLUNAS)
    aux['nivel_hsk'] = hsk
    df_list.append(aux.copy())
df = pd.concat(df_list).reset_index(drop=True)
df

,simplificado,tradicional,pinyin_num,pinyin_carac,significado,nivel_hsk
0,我,我,wo3,wǒ,I; me,1
1,你,你,ni3,nǐ,you (singular),1
2,是,是,shi4,shì,be; is; are; am,1
3,了,了,le5,le,indicates a completed or finished action,1
4,不,不,bu4,bù,no; not,1
...,...,...,...,...,...,...
4990,深情厚谊,深情厚誼,shen1qing2hou4yi4,shēnqínghòuyì,profound friendship,6
4991,武侠,武俠,wu3xia2,wǔxiá,knight-errant; a genre of swordplay martial ar...,6
4992,将就,將就,jiang1jiu5,jiāngjiu,put up with; accept somewhat reluctantly,6
4993,对联,對聯,dui4lian2,duìlián,rhyming couplet; vertical written couplet usua...,6


In [23]:
palavras = df[['simplificado','nivel_hsk']]

In [27]:
ideo = list()
for i in range(0, palavras['simplificado'].str.len().max()):
    ideo.append(palavras['simplificado'].str[i].dropna())
ideo

lista = pd.Series()
for i in ideo:
    lista = lista.append(i)
lista

<ipython-input-27-3eb6f8d3eccc>:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lista = pd.Series()


0       我
1       你
2       是
3       了
4       不
       ..
4982    见
4983    深
4987    桑
4988    兽
4990    谊
Length: 9656, dtype: object

In [28]:
lista[lista == '上']

28      上
142     上
259     上
569     上
2293    上
3220    上
3323    上
4073    上
4246    上
4635    上
213     上
234     上
360     上
4404    上
4623    上
4621    上
dtype: object

In [29]:
lista = lista.reset_index(drop=True)

In [32]:
df_qtd = pd.DataFrame()
df_qtd['caracter'] = lista
df_qtd = df_qtd.groupby('caracter').size().reset_index(name='qtd')
df_qtd = df_qtd.sort_values('qtd', ascending=False)
df_qtd

,caracter,qtd
8,不,64
659,子,58
875,心,41
1388,气,39
373,发,38
...,...,...
869,徘,1
870,徙,1
871,御,1
1768,磕,1


In [34]:
df_qtd.to_csv('../data/ideogramas por quantidade.txt', index=False)